# <center>Structural Analysis and Visualization of Networks</center>

## <center/>Course Project #2

### <center>Student: *Nazarov Ivan*</center>

#### <hr /> General Information

**Hard Deadline:** 21.06.2015 23:59 <br \>

Please send your reports to <mailto:leonid.e.zhukov@gmail.com> and <mailto:shestakoffandrey@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2015] *Nazarov* *Ivan* Project*2***

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

# Description

## Task 1 

You are provided with the [DBLP dataset](https://www.dropbox.com/s/ft4ekv2f3r43u7b/dblp_2000.csv.gz?dl=0) (warning, raw data!). It contains coauthorships that were revealed during $2000$-$2014$. Particularly, the file contains $3$ colomns: first two for authors' names and the third for the year of publication. This data can be naturally mapped to undirected graph structure.

Your task is construct supervised link prediction scheme.

### Guidelines:

0. Use *pandas* module to load and manipulate the dataset in Python
1. Initiallize your classification set as follows:
    * Determine training and testing intervals on your time domain (for instance, in DBLP dataset take a period $2000$-$2010$ as training period and $2011$-$2014$ as testing period)
    * Pick pairs of authors that **have appeared during training interval** but **have not published together** during it
    * These pairs form **positive** or **negative** examples depending on whether they have formed coauthorships **during the testing interval**
    * You have arrived to binary classification problem. PROFIT!
2. Construct feature space:
    * Most of our features tend to be topological. Examples of the features can be: (weighted) sum of neigbours, shortest distance, etc
3. Choose at least $4$ classification algorithms from [scikit module](http://scikit-learn.org/stable/) (goes with Anaconda) and compare them in terms of Accuracy, Precision, Recall, F-Score (for positive class) and Mean Squared Error. Use k-fold cross-validation and average your results

In [1]:
import pandas as pd, numpy as np, scipy.sparse as sp
import os, gc, regex as re, time as tm

import matplotlib.pyplot as plt
%matplotlib inline

DATADIR = os.path.realpath( os.path.join( ".", "data", "proj02" ) )

raw_dblp_file = os.path.join( DATADIR, "dblp_2000.csv.gz" )
cached_dblp_file = os.path.join( DATADIR, "dblp_2000.ppdf" )
cached_author_index = os.path.join( DATADIR, "dblp_2000_authors.txt" )

Define some helper functions

In [2]:
## Return a mask of elements of b found in a: optimal for numeric arrays
def match( a, v, return_indices = False ) :
	index = np.argsort( a )
## Get insertion indices
	sorted_index = np.searchsorted( a, v, sorter = index )
## Truncate the indices by the length of a
	index = np.take( index, sorted_index, mode = "clip" )
	mask = a[ index ] == v
## return
	if return_indices :
		return mask, index[ mask ]
	return mask

A handy procedure for converting an $(v_{ij})$ list into a sparse matrix.

In [3]:
## Convert the edgelist into sparse matrix
def to_sparse_coo( u, v, shape, dtype = np.int32 ) :
## Create a COOrdinate sparse matrix from the given ij-indices
	assert( len( u ) == len( v ) )
	return sp.coo_matrix( (
			np.ones( len( u ) + len( v ), dtype = dtype ), (
				np.concatenate( ( u, v ) ), np.concatenate( ( v, u ) ) )
		), shape = shape )

## Remeber: when converting COO to CSR/CSC the duplicate coordinate
##  entries are summed!

Load the DBLP dataset, making a cached copy if required.

In [4]:
## Create cache if necessary
tick = tm.time( )
if not os.path.exists( cached_dblp_file ) :
	## Load the csv file into a dataframe
	dblp = pd.read_csv( raw_dblp_file, # nrows = 10000,
	## On-the-fly decompression
			compression = "gzip", header = None, quoting = 0,
	## Assign column headers
			names = [ 'author1', 'author2', 'year', ], encoding = "utf-8" )
	## Finish
	tock = tm.time( )
	print "Raw DBLP read in %.3f sec." % ( tock - tick, )
## Map author names to ids
	from sklearn.preprocessing import LabelEncoder
	le = LabelEncoder( ).fit( np.concatenate( (
		dblp["author1"].values, dblp["author2"].values, ) ) )
	dblp_author_index = le.classes_
	for col in [ 'author1', 'author2', ] :
		dblp[ col ] = le.transform( dblp[ col ] )
## Cache
	dblp.to_pickle( cached_dblp_file )
	with open( cached_author_index, "w" ) as out :
		for label in le.classes_ :
			out.write( label.strip( ).encode( "utf-8" ) + "\n" )
	del dblp, le
## Finish
	tick = tm.time( )
	print "Preprocessing took %.3f sec." % ( tick - tock, )
else :
## Load the database from pickled format
	dblp = pd.read_pickle( cached_dblp_file )
## Read the dictionary of authors
	with open( cached_author_index, "r" ) as author_index :
		dblp_author_index = [ line.decode( "utf-8" ) for line in author_index ]
## Report
tock = tm.time( )
print "DBLP loaded in %.3f sec." % ( tock - tick, )

DBLP loaded in 15.734 sec.


Now split the DBLP dataset in two periods: pre and post 2010.

First preprocess the pre 2010 data.

In [5]:
pre2010 = dblp[ dblp.year <= 2010 ].copy( )

Reencode the vertices of the pre-2010 graph in a less wasteful format. Use sklearn's LabelEncoder() to this end.

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder( ).fit( np.concatenate( ( pre2010[ "author1" ].values, pre2010[ "author2" ].values ) ) ) 
pre2010_values = le.classes_

## Recode the edge data
for col in [ 'author1', 'author2', ] :
    pre2010[ col ] = le.transform( pre2010[ col ] )      

Convert the edge list data into a sparse matrix

In [7]:
pre2010_adj = to_sparse_coo(
    pre2010[ "author1" ].values, pre2010[ "author2" ].values,
    shape = 2 * [ len( le.classes_ ) ] )

## Eliminate duplicates by converting them into ones
pre2010_adj = pre2010_adj.tocsr( )
pre2010_adj.data = np.ones_like( pre2010_adj.data )

Find the vertices of the pre 2010 period that are in post-2010

In [8]:
post2010 = dblp[ dblp.year > 2010 ]
common_vertices = np.intersect1d( pre2010_values,
	np.union1d( post2010[ "author1" ].values, post2010[ "author2" ].values ) )

Remove completely new vertices from post 2010 data

In [9]:
post2010 = post2010[ (
    match( common_vertices, post2010[ "author1" ].values ) &
    match( common_vertices, post2010[ "author2" ].values ) ) ]
del common_vertices

Map the post 2010 vertices to pre 2010 vertices and construct the adjacency matrix.

In [10]:
for col in [ 'author1', 'author2', ] :
    post2010[ col ] = le.transform( post2010[ col ] )

## The adjacency matrix
post2010_adj = sp.coo_matrix( ( np.ones( post2010.shape[0], dtype = np.bool ),
        ( post2010[ "author1" ].values, post2010[ "author2" ].values )
    ), shape = pre2010_adj.shape ).tolil( )

Leave only those edges in the post 2010 dataset, which had not existed during 2000-2010.

In [11]:
post2010_adj[ pre2010_adj.nonzero( ) ] = 0

Eleminate duplicate edges and transform into a CSR format

In [12]:
post2010_adj = post2010_adj.tocsr( )
post2010_adj.data = np.ones_like( post2010_adj.data )

Here we have two aligned symmetric adjacency matrices : for edges exisited before 2010 and new edges formed after 2010.

In [13]:
print post2010_adj.__repr__()
print pre2010_adj.__repr__( )

<873881x873881 sparse matrix of type '<type 'numpy.bool_'>'
	with 805815 stored elements in Compressed Sparse Row format>
<873881x873881 sparse matrix of type '<type 'numpy.int32'>'
	with 6518435 stored elements in Compressed Sparse Row format>


All edes of the post2010 graph are included and considered to be positive examples

In [14]:
positive = np.append( *( c.reshape((-1, 1)) for c in post2010_adj.nonzero( ) ), axis = 1 )

Now a slightly harder part is to generate an adequate number of negative examples, so that the final training sample would be balanced.

In [15]:
## Generate a sample of vertex pairs with no edge in both periods
negative = np.random.choice( pre2010_adj.shape[ 0 ], size = ( 2 * positive.shape[0], positive.shape[1] ) )

Compie the final training dataset.

In [16]:
E = np.vstack( ( positive, negative ) )
y = np.vstack( ( np.ones( ( positive.shape[ 0 ], 1 ), dtype = np.float ),
                np.zeros( ( negative.shape[ 0 ], 1 ), dtype = np.float ) ) )

So, finally, we got ourselves a trainig set of edges with 2:1 negative-to-postive ratio.

## Feature construction

The first pair of features is the degrees on the edge endpoints: for $(i,j) \in V\times V$
$$ \phi^1_{ij} = |N_i|\,\text{ and }\,\phi^2_{ij} = |N_j|\,,$$
where $N_v$ is the set of adjacent vertices of a node $v$.

In [17]:
def phi_degree( edges, A ) :
	deg = A.sum( axis = 1 ).astype( np.float )
	return np.append( deg[ edges[ :, 0 ] ], deg[ edges[ :, 1 ] ], axis = 1 )

It turns out that at least two edge features can be constructed via a so called "sandwich" matrix.

In [18]:
def __sparse_sandwich( edges, A, W = None ) :
    AA = A.dot( A.T ) if W is None else A.dot( W ).dot( A.T )
    result = AA[ edges[:,0], edges[:,1] ]
    del AA ; gc.collect( 0 ) ; gc.collect( 1 ) ; gc.collect( 2 )
    return result.reshape(-1, 1)

The next geature is the Adamic/adar score: for $(i,j)\in V\times V$
$$ \phi^3_{ij} = \sum_{v\in N_i \cap N_j } \frac{1}{\log |N_v|}\,.$$

Another feature is the numberod neighbours shared by the endpoints:
$$\phi^4_{ij} = |N_i\cap N_j|\,.$$

In fact both features are special cases of the same formula :
$$ (\phi_{ij}) = A W A'\,, $$
where $W$ is the weight matrix. In the case of common neigbours it is the unit matrix $I$, whereas for Adamic/Adar it is the diagonal matrix of reciprocal of degree logarithms :
$$ W = \text{diag}\Bigl( \frac{1}{\log |N_i|} \Bigr)_{i\in V}\,.$$

In [19]:
def phi_adamic_adar( edges, A ) :
    inv_log_deg = 1.0 / np.log( A.sum( axis = 1 ).getA1( ) )
    inv_log_deg[ np.isinf( inv_log_deg ) ] = 0
    result = __sparse_sandwich( edges, A, sp.diags( inv_log_deg, 0 ) )
    del inv_log_deg ; gc.collect( 0 ) ; gc.collect( 1 ) ; gc.collect( 2 )
    return result

In [20]:
def phi_common_neighbours( edges, A ) :
    return __sparse_sandwich( edges, A )

## Computing the features

Vertex degrees

In [21]:
tick = tm.time()
phi_12 = phi_degree( E, pre2010_adj )
tock = tm.time()
print "Vertex degree computed in %.3f sec." % ( tock - tick, )

Vertex degree computed in 0.561 sec.


Adamic/Adar metric

In [22]:
tick = tm.time()
phi_3 = phi_adamic_adar( E, pre2010_adj )
tock = tm.time()
print "Adamic/adar computed in %.3f sec." % ( tock - tick, )

Adamic/adar computed in 169.297 sec.


Common neighbours

In [23]:
tick = tm.time()
phi_4 = phi_common_neighbours( E, pre2010_adj )
tock = tm.time()
print "Common neighbours computed in %.3f sec." % ( tock - tick, )

Common neighbours computed in 60.207 sec.


Compute all-pairs shortest paths

In [24]:
# tick = tm.time()
# phi_5 = phi_shortest_paths( E, pre2010_adj )
# tock = tm.time()
# print "Shortest paths computed in %.3f sec." % ( tock - tick, )

Collect all features into a numpy matrix

In [25]:
X = np.hstack( ( phi_12, phi_3, phi_4 ) )

Having computed all the features, lets make a subsample so that the classfification would run faster.

In [26]:
from sklearn.cross_validation import train_test_split

In [27]:
X_modelling, X_main, y_modelling, y_main = train_test_split( X, y.ravel( ), train_size = 0.20 )

Attach SciKit's grid search and x-validation modules.

In [28]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score

We are going to analyze many classifiers at once.

In [29]:
classifiers = list( )

### Logistic Regression

Logistic regression for binary classification solves the following problem on the training dataset $(x_i,t_i)_{i=1}^n \in \mathbb{R}^{1+p}\times \{0,1\}$:
$$ \sum_{i=1}^n t_i \log \sigma( \beta'x_i ) + (1-t_i) \log \bigl( 1-\sigma( \beta'x_i ) \bigr) \to \min_{\beta, \beta_0} \,, $$
where $\sigma(z) = \bigr(1+e^{-z}\bigl)^{-1}$. The classification is done using the folowing rule :  
$$ \hat{t}(x) = \mathop{\text{argmax}}_{k=1,2}\, \mathbb{P}(T=k|X=x)\,, $$
where $\mathbb{P}(T=1|X=x) = \sigma(\beta'x)$.

In [30]:
from sklearn.linear_model import LogisticRegression

LR_grid = GridSearchCV( LogisticRegression( ), cv = 10, verbose = 1,
        param_grid = { "C" : np.logspace( -2, 2, num = 5 ) }, n_jobs = -1
    ).fit( X_modelling, y_modelling )

classifiers.append( ( "Logistic", LR_grid.best_estimator_ ) )

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  48 out of  50 | elapsed:  3.0min remaining:    7.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.1min finished


### Linear and Quadratic Discriminant Analysis

It is a widely known fact that sometimes simple models beat more complicated ones in terms of their accuracy. Thus let's consdier LDA and QDA.

In [31]:
from sklearn.lda import LDA
from sklearn.qda import QDA

classifiers.append( ( "LDA", LDA( ) ) )
classifiers.append( ( "QDA", QDA( ) ) )

### Decision tree classifiers

Let's employ the classification tree model. On its own a decision tree is a volatile classifier, meaning that the addition of new data can drammatically alter its structure, that is why let's utilize boosted trees and randomized forests. These methods learn the intirinsic nonlinear features of the data by iterative construction of weak classifiers focusing on different aspects of the dataset.

#### Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

RF_grid = GridSearchCV( RandomForestClassifier( n_estimators = 50 ), cv = 10, verbose = 1,
        param_grid = { "max_depth" : [ 3, 5, 15, 30 ] }, n_jobs = -1
    ).fit( X_modelling, y_modelling )

classifiers.append( ( "RandomForest", RF_grid.best_estimator_ ) )

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:   36.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 19.9min finished


#### Boosted tree (AdaBoost)

In [34]:
from sklearn.ensemble import AdaBoostClassifier

classifiers.append( ( "AdaBoost", AdaBoostClassifier( n_estimators = 50 ) ) )

#### Simple tree

One does not expect a simple tree to do comparably well against ensemble classifiers.

In [35]:
from sklearn.tree import DecisionTreeClassifier

tree_grid = GridSearchCV( DecisionTreeClassifier( criterion = "gini" ), cv = 10, verbose = 1,
        param_grid = { "max_depth" : [ 3, 5, 15, 30 ] }, n_jobs = -1
    ).fit( X_modelling, y_modelling )

classifiers.append( ( "Tree", tree_grid.best_estimator_ ) )

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:  1.0min remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min finished


### $k$-Nearest Negihbours

Another quite engineering approach to classification is to follow a simple rule : if the majority of a points $l$ nearest neighbours correspond to a class $c$ then this point is very likely to come from calss $c$ as well. Know them by their freinds!

In [36]:
from sklearn.neighbors import KNeighborsClassifier

knn_grid = GridSearchCV( KNeighborsClassifier(  ), cv = 10, verbose = 1,
        param_grid = { "n_neighbors" : [ 3, 5, 15, 30 ] }, n_jobs = -1
    ).fit( X_modelling, y_modelling )

classifiers.append( ( "k-NN", knn_grid.best_estimator_ ) )

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed: 42.6min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 44.3min finished


### Support Vector Machine classification

In [37]:
from sklearn.svm import SVC

In [38]:
from sklearn.linear_model import SGDClassifier

## Testing

Split the dataset into train and test.

In [39]:
X_train, X_test, y_train, y_test = train_test_split( X_main, y_main, train_size = 0.20 )

Subsample the train dataset

In [44]:
subsample = np.random.permutation( X_train.shape[ 0 ] )#[ : 50000 ]
X_train_subsample, y_train_subsample = X_train[ subsample ], y_train[ subsample ]

Run tests

In [45]:
results = dict()
for name, clf in classifiers :
    tick = tm.time( )
    results[name] = cross_val_score( clf, X_train_subsample, y_train_subsample, n_jobs = -1, verbose = 1, cv = 10 )
    tock = tm.time( )
    print "k-fold crossvalidation for %s took %.3f sec." % ( name, tock - tick, )

[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:   29.1s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   34.8s finished


k-fold crossvalidation for Logistic took 36.091 sec.


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    7.3s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    8.3s finished


k-fold crossvalidation for LDA took 10.003 sec.


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    6.2s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    7.2s finished


...

k-fold crossvalidation for AdaBoost took 177.608 sec.


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:   11.2s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.2s finished


k-fold crossvalidation for Tree took 14.085 sec.


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  7.3min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.8min finished


k-fold crossvalidation for k-NN took 526.373 sec.


In [46]:
k_fold_frame = pd.DataFrame( results )

In [48]:
# k_fold_frame.append( k_fold_frame.apply( np.average ), ignore_index = True )
k_fold_frame.apply( np.average )

AdaBoost        0.834854
LDA             0.751858
Logistic        0.823119
QDA             0.788113
RandomForest    0.835156
Tree            0.832196
k-NN            0.832491
dtype: float64

## Task 2

Consider the [flickr dataset](https://www.dropbox.com/s/srsib3hq863drtp/flickr_data.tar.gz?dl=0) (warning, raw data!). <br/>
File ''*users.txt*'' provides a table of form *userID*, *enterTimeStamp*, *additionalInfo*... <br/>
File "*contacts.txt*" consists of pairs of *userID*'s and link establishment timestamp <br/>

Recall *scoring functions* for link prediction. Your task is to compare the performance of each scoring function as follows:
1. TOP-$n$ accuracy
    * Denote the number of links $E_\text{new}$ appeared during testing period as $n$
    * Denote the ranked list of node pairs provided by score $s$ as $\hat{E}_s$
    * Take top-$n$ pairs from $\hat{E}_s$ and intersect it with $E_\text{new}$. Performance is measured as the size of resulted set
2. ROC and AUC ('star' subtask)

Essentially, for this task you also have to follow the guideline points $1$ and $2$ above. The only thing you have to keep in mind is that flickr dataset is growing dataset. Since then, consider nodes that are significantly represented both in training and testing intervals (for instance, have at least $5$ adjacent edges in training and testing intervals)